In [1]:
# This program aims to determine the oligo sequence matching to the keyword. 
# The length of a keyword is typically 6 symbols (48 bit in ASCII code), therefore the converted oligo sequence should have 21 nt, which coincides with the length of recognition domain in a crRNA.

keyword = 'CRISPR'
my_keyword_hexdata = keyword.encode("utf-8").hex()

scale = 16 
num_of_bits = 8

# Pre-processing the binary data to make the length a multiple of 4
a = bin(int(my_keyword_hexdata, scale))[2:].zfill(num_of_bits)
if (len(a) % 4 != 0):
    a = '0' * (4-(len(a)%4)) + a


import math
from array import *
import collections

%store -r Groups

baseMappingMatrix = ['ACGT', 'CGTA', 'GTAC', 'TACG']

def ternary(n):
    if n == 0:
        return '0'
    nums = []
    while n:
        n, r = divmod(n, 3)
        nums.append(str(r))
    return ''.join(reversed(nums))
    
def base3ToOligo(seq):
    base3ToOligo = {'0': 'G', '1': 'T', '2': 'A'}
    base3ToOligoConverted = []
    for i in range (0, len(seq)):
        base3ToOligoConverted.append(base3ToOligo[seq[i]])
        if base3ToOligo[seq[i]] == 'C':
            base3ToOligo = {'0': 'G', '1': 'T', '2': 'A'}
        elif base3ToOligo[seq[i]] == 'G':
            base3ToOligo = {'0': 'T', '1': 'A', '2': 'C'}
        elif base3ToOligo[seq[i]] == 'T':
            base3ToOligo = {'0': 'A', '1': 'C', '2': 'G'}
        elif base3ToOligo[seq[i]] == 'A':
            base3ToOligo = {'0': 'C', '1': 'G', '2': 'T'}
    return ''.join(base3ToOligoConverted)

originalData = a

originalDataList = []
dataPositions = []
dataPositionsUnits = []

arrayGap = 16

# Get Combinations
addedNumber = ((int(len(originalData)/arrayGap)) * arrayGap) - len(originalData)
correctedData = originalData + '0'*addedNumber
arraySegmentNumber = int(len(correctedData)/arrayGap)
for j in range (0, arraySegmentNumber):
    originalDataSegment = correctedData[arrayGap*j:arrayGap*(j+1)]
    originalDataList.append(originalDataSegment)
    for k in range (0, arrayGap):
        if (originalDataSegment[k] == '1'):
            dataPositionsUnits.append(k)
    dataPositions.append(dataPositionsUnits)
    
    dataPositionsUnits = []
    
print('Combinations of the query: ' + str(dataPositions))

# Get 7-nt Query Bases for each two symbols (16-bit) through referencing the Groups
firstIndex = []
dataSequence = []
for i in range (0, len(dataPositions)):
    for j0 in range (0, len(Groups)):
        for j1 in range (0, len(Groups[j0])):
            if dataPositions[i] == Groups[j0][j1]:
                firstIndex.append(j0)
                ternaryNumber = ternary(j0)
                ternaryNumber = '0'*(6-len(ternaryNumber)) + ternaryNumber
                dataSequence.append(base3ToOligo(ternaryNumber))
                dataSequence.append(baseMappingMatrix[j1][j0%4])
                
print('Indexes: ' + str(firstIndex))
dataSequence = ''.join(dataSequence)  # Put together Query Bases to make a 21-nt oligo sequence
print('Query Sequence: ' + str(dataSequence))


Combinations of the query: [[1, 6, 7, 9, 11, 14], [1, 4, 7, 9, 11, 14, 15], [1, 3, 9, 11, 14]]
Indexes: [716, 650, 641]
Query Sequence: ATGAGCCATACGCTAGCACAG
